In [7]:
import math
# from timber.trainer.timber_trainer import load_model
import torch
import os

_N_H = 3
TDST_BQ = 5
MASK_K_BK = 4
MODEL_N = 3

ensemble_attn_mask_per_layer = torch.randint(1, 11, (_N_H, TDST_BQ, MASK_K_BK, MODEL_N))

In [8]:
ensemble_attn_mask_per_layer = ensemble_attn_mask_per_layer.view(_N_H * TDST_BQ, MASK_K_BK * MODEL_N)
# ensemble_attn_mask_per_layer : [N*H * TDST//BLOCK_SIZE_Q, mask_k//BLOCK_SIZE_K * ensemble_model_n]
unique_x, indices, unique_cnt = torch.unique(ensemble_attn_mask_per_layer, return_inverse=True, sorted=False, return_counts=True)
# indices -= indices.min(dim=1, keepdims=True)[0]

# cnt_x = (unique_x[None, None, :] == ensemble_attn_mask_per_layer[:, :, None]).long().sum(1)
N, K = ensemble_attn_mask_per_layer.shape

cnt_xs = []
chunk_n = 64
for icn in range(int(math.ceil(N / chunk_n))):
    cnt_xs.append(
        (unique_x[None, None, :] == ensemble_attn_mask_per_layer[icn*chunk_n:icn*chunk_n+chunk_n, :, None]).long().sum(1)
    )
cnt_x = torch.cat(cnt_xs, dim=0)


result = torch.full_like(ensemble_attn_mask_per_layer, 9999999)
result = result.scatter_(1, indices.clamp(0, K-1), ensemble_attn_mask_per_layer)


# t = result[:, :, None] == unique_x[None, None, :]
# t = t * torch.arange(len(unique_x), device=t.device)[None, None, :]
# t = t.sum(-1)
N, K = result.shape
ts = []
chunk_n = 64
for icn in range(int(math.ceil(N / chunk_n))):
    t = result[icn*chunk_n:icn*chunk_n+chunk_n, :, None] == unique_x[None, None, :]
    t = t * torch.arange(len(unique_x), device=t.device)[None, None, :]
    t = t.sum(-1)
    ts.append(t)
t = torch.cat(ts, dim=0)

# torch.Size([4096, 1280]) torch.Size([64, 1280]) torch.Size([4096, 2094])
# print(result.shape, t.shape, cnt_x.shape)
result_cnt = torch.where(result < 9999999, cnt_x.gather(-1, t), -9999999)

In [9]:
ensemble_attn_mask_per_layer

tensor([[ 3,  3,  4,  8,  8,  9,  4,  9,  9,  4,  7,  2],
        [ 6, 10,  8,  4,  1,  9,  7, 10, 10,  4,  2,  4],
        [ 9,  8,  3,  7,  9,  5,  2, 10,  2,  4,  1,  8],
        [ 1,  6,  8,  4, 10,  9,  4,  1,  4,  2, 10,  9],
        [ 3,  6,  5,  7,  7,  2,  8,  2,  8,  7, 10,  8],
        [ 9,  4,  2,  8,  1,  7,  4,  7,  8,  5,  2, 10],
        [10,  7,  2,  1,  2,  9,  4,  3, 10,  4,  1,  9],
        [ 1,  6,  6,  6,  6,  4,  2,  7, 10,  9,  2,  9],
        [ 3,  5,  2,  9,  6,  5,  3,  2,  5,  6,  6,  9],
        [10,  4,  4,  2,  2,  8,  9,  8,  9,  3,  4,  9],
        [ 6,  8,  8,  4,  9,  8,  9,  1,  9, 10,  6,  3],
        [ 8,  8,  6,  8,  4,  9,  4,  1, 10,  4,  5,  3],
        [ 4, 10,  8,  8,  9,  8,  4,  2,  8,  3,  7, 10],
        [10,  3,  5,  4,  6,  6,  6, 10,  5, 10,  6,  9],
        [ 9,  8,  7,  9,  7,  3,  6,  1, 10,  1,  1,  8]])

In [10]:
result

tensor([[9999999,       2,       3,       4, 9999999, 9999999,       7,       8,
               9, 9999999, 9999999, 9999999],
        [      1,       2, 9999999,       4, 9999999,       6,       7,       8,
               9,      10, 9999999, 9999999],
        [      1,       2,       3,       4,       5, 9999999,       7,       8,
               9,      10, 9999999, 9999999],
        [      1,       2, 9999999,       4, 9999999,       6, 9999999,       8,
               9,      10, 9999999, 9999999],
        [9999999,       2,       3, 9999999,       5,       6,       7,       8,
         9999999,      10, 9999999, 9999999],
        [      1,       2, 9999999,       4,       5, 9999999,       7,       8,
               9,      10, 9999999, 9999999],
        [      1,       2,       3,       4, 9999999, 9999999,       7, 9999999,
               9,      10, 9999999, 9999999],
        [      1,       2, 9999999,       4, 9999999,       6,       7, 9999999,
               9,      10, 999

In [11]:
result_cnt

tensor([[-9999999,        1,        2,        3, -9999999, -9999999,        1,
                2,        3, -9999999, -9999999, -9999999],
        [       1,        1, -9999999,        3, -9999999,        1,        1,
                1,        1,        3, -9999999, -9999999],
        [       1,        2,        1,        1,        1, -9999999,        1,
                2,        2,        1, -9999999, -9999999],
        [       2,        1, -9999999,        3, -9999999,        1, -9999999,
                1,        2,        2, -9999999, -9999999],
        [-9999999,        2,        1, -9999999,        1,        1,        3,
                3, -9999999,        1, -9999999, -9999999],
        [       1,        2, -9999999,        2,        1, -9999999,        2,
                2,        1,        1, -9999999, -9999999],
        [       2,        2,        1,        2, -9999999, -9999999,        1,
         -9999999,        2,        2, -9999999, -9999999],
        [       1,        2

In [12]:
ensemble_cnt_sorted, indices = torch.sort(result_cnt, dim=-1, descending=True)
ensemble_sorted = result.gather(-1, indices)
# mask = ensemble_cnt_sorted >= ensemble_method_final_inter_thresh

# ensemble_filtered = torch.where(mask, ensemble_sorted, torch.tensor(9999999, device=mask.device))
# ensemble_cnt_filtered = torch.where(mask, ensemble_cnt_sorted, torch.tensor(9999999, device=mask.device))


In [13]:
ensemble_sorted

tensor([[      4,       9,       3,       8,       2,       7, 9999999, 9999999,
         9999999, 9999999, 9999999, 9999999],
        [      4,      10,       1,       2,       6,       7,       8,       9,
         9999999, 9999999, 9999999, 9999999],
        [      2,       8,       9,       1,       3,       4,       5,       7,
              10, 9999999, 9999999, 9999999],
        [      4,       1,       9,      10,       2,       6,       8, 9999999,
         9999999, 9999999, 9999999, 9999999],
        [      7,       8,       2,       3,       5,       6,      10, 9999999,
         9999999, 9999999, 9999999, 9999999],
        [      2,       4,       7,       8,       1,       5,       9,      10,
         9999999, 9999999, 9999999, 9999999],
        [      1,       2,       4,       9,      10,       3,       7, 9999999,
         9999999, 9999999, 9999999, 9999999],
        [      6,       2,       9,       1,       4,       7,      10, 9999999,
         9999999, 9999999, 999

In [14]:
ensemble_cnt_sorted

tensor([[       3,        3,        2,        2,        1,        1, -9999999,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       3,        3,        1,        1,        1,        1,        1,
                1, -9999999, -9999999, -9999999, -9999999],
        [       2,        2,        2,        1,        1,        1,        1,
                1,        1, -9999999, -9999999, -9999999],
        [       3,        2,        2,        2,        1,        1,        1,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       3,        3,        2,        1,        1,        1,        1,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       2,        2,        2,        2,        1,        1,        1,
                1, -9999999, -9999999, -9999999, -9999999],
        [       2,        2,        2,        2,        2,        1,        1,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       4,        2